# 20220601 NEW VERSION

In [1]:
##做法:
'''
1. 去pdf檔案中找到所以的資訊(表格名稱)，如果已存在程式碼中則不需要改變，但如果不再程式碼中需要新增
2. 新增的步驟是 先找到PDF中明確的表格名稱，將擁有這個表格名稱的內容進行文字取代，取代為你想要的表格名稱，再將這個表格名稱加入LIST內
3. 每個都完成上述1,2步驟後，一路直行到底，然後去final_check.csv這個檔案看他有沒有抓到所以得新資料
4. 去 下半部有old標記無可能以刪除的.csv這個檔案看看最後一欄有old標記的是否真的已刪除
5. 去 page3.csv的檔案將['category','investment Type','Asset Class']欄位為空白的補上，這些代表是新增的
'''

"\n1. 去pdf檔案中找到所以的資訊(表格名稱)，如果已存在程式碼中則不需要改變，但如果不再程式碼中需要新增\n2. 新增的步驟是 先找到PDF中明確的表格名稱，將擁有這個表格名稱的內容進行文字取代，取代為你想要的表格名稱，再將這個表格名稱加入LIST內\n3. 每個都完成上述1,2步驟後，一路直行到底，然後去final_check.csv這個檔案看他有沒有抓到所以得新資料\n4. 去 下半部有old標記無可能以刪除的.csv這個檔案看看最後一欄有old標記的是否真的已刪除\n5. 去 page3.csv的檔案將['category','investment Type','Asset Class']欄位為空白的補上，這些代表是新增的\n"

In [2]:
import pandas as pd
import pdfplumber as pr
import re
import os
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
PDF_files = []
input_path = r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input'
for x in os.listdir(input_path):
    if '.pdf' and '年4月' in x:
        PDF_files.append(x)
#這個檔案不要刪除，很重要
GFI = pd.read_excel(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input/202204政府基金運用資訊 v2.0.xlsx',sheet_name = 'Offshore (Raw data)',skiprows=1)

In [4]:
#勞退新制
LPF_LIST_ORI = GFI[GFI['Fund']=='LPF']['Mandate Type'].unique()
#勞退舊制
LRF_LIST_ORI = GFI[GFI['Fund']=='LRF']['Mandate Type'].unique()
#勞保
LIF_LIST = GFI[GFI['Fund']=='LIF']['Mandate Type'].unique()
#國保
NPIF_LIST = GFI[GFI['Fund']=='NPIF']['Mandate Type'].unique()
#退撫
PSPF_LIST = GFI[GFI['Fund']=='PSPF']['Mandate Type'].unique()

### LPF LRF

In [59]:
勞退_PDF = pr.open(input_path+'/勞工退休基金-111年4月.pdf')
#從0開始
#是一個list
ps              = 勞退_PDF.pages
#用range配len跑for迴圈
pg = ps[23]
page_text = pg.extract_text()
print(page_text)
print('100-1 全球基本面指數被動股票型(續約 2) ' in page_text)

              98-1 亞太(日本除外)主動股票型(續約 2)    
單位：美元 
委任迄今 
111年截至4月  (106.11.17~111.4.30) 
受託機構  委託金額  目前淨資產  累積投資報酬 投資 
指標 
(％)  報酬率 排名 
報酬率(％) 
(％) 
景順  307,323,526   360,272,255   -2.34  21.97  －  11.76 
 
             100-1 全球基本面指數被動股票型(續約 2)       
單位：美元
委任迄今 
111年截至4月  (111.3.6~111.4.30)    
受託機構  委託金額  目前淨資產  累積投資報酬率 投資  指標 
(％)  報酬率 排名  報酬率
(％)  (％) 
貝萊德   304,997,166   301,353,972   -1.19  -1.19  2 
道富   360,436,214   355,564,428   -1.35  -1.35  3 
瑞銀   308,879,836   305,714,958   -1.02  -1.02  1  -1.57 
Nomura   360,147,608   354,968,110   -1.44  -1.44  4 
合計  1,334,460,824  1,317,601,468   -1.26   -1.26   － 
 
 
 
                101-1 全球低波動指數被動股票型(續約)    
  單位：美元 
委任迄今 
111年截至4月  (107.6.13~111.4.30) 
受託機構  委託金額  目前淨資產  累積投資報酬率 投資  指標 
(％)  報酬率 排名  報酬率
(％)  (％) 
貝萊德  671,455,711  797,971,173  -5.96  18.84  －  18.12 
   
 
 
 
 
 
  24 
True


In [10]:
PDF_files

['勞工保險基金-111年4月.xlsx',
 '勞工退休基金-111年4月.pdf',
 '勞工保險基金-111年4月.pdf',
 '勞工退休基金-111年4月.xlsx',
 '國民年金保險基金-111年4月.xlsx',
 '111年4月基金運用情形.xlsx',
 '國民年金保險基金-111年4月.pdf',
 '111年4月基金運用情形.pdf']

In [61]:
"""
有特殊情況，新舊制再同一頁，之後要定期看有沒有需要更改
row 177 , 208的檔案路徑有可能需要更改
"""


LPF_LIST = []
LRF_LIST = []
有新制勞退目標基金的頁數 = []
新制勞退目標基金的名稱   = []
NEW_FUND_NAMES         = []

有舊制勞退目標基金的頁數 = []
舊制勞退目標基金的名稱   = []
OLD_FUND_NAMES         = []

for PDF_f in PDF_files:
    if '勞工退休基金' in PDF_f and '.pdf' in PDF_f:
        勞退_PDF = pr.open(input_path+'/'+PDF_f)
        #從0開始
        #是一個list
        ps              = 勞退_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            if p < 22:
                #新制，將PDF檔案中的基金名稱改成excel內的
                FUND_NAME = 'LPF'
                if '97-1 全球主動債券型（續約 3）' in page_text:
                    print('97-1 全球主動債券型（續約 3） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-1 全球主動債券型（續約 3）','97-1 全球債券型（ 續約3）')
                    LPF_LIST.append('97-1 全球債券型（ 續約3）')
                if '97-2 全球增值債券型（續約 3）' in page_text:
                    print('97-2 全球增值債券型（續約 3） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-2 全球增值債券型（續約 3）','97-2 全球增值債券型（續約 3）')
                    LPF_LIST.append('97-2 全球增值債券型（續約 3）')
                if '97-2 全球增值股票型（續約 3）' in page_text:
                    print('97-2 全球增值股票型（續約 3） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-2 全球增值股票型（續約 3）','97-2 全球增值股票型（續約 3）')
                    LPF_LIST.append('97-2 全球增值股票型（續約 3）')
                if '98-1 亞太（日本除外）主動股票型（續約 2）' in page_text:
                    print('98-1 亞太（日本除外）主動股票型（續約 2） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('98-1 亞太（日本除外）主動股票型（續約 2）','98-1 亞太（ 日本除外）股票型（ 續約2）')
                    LPF_LIST.append('98-1 亞太（ 日本除外）股票型（ 續約2）')
                if '98-1 全球被動股票型（續約 2）' in page_text:
                    print('98-1 全球被動股票型（續約 2） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('98-1 全球被動股票型（續約 2）','98-1 全球被動股票型（ 續約2）')
                    LPF_LIST.append('98-1 全球被動股票型（ 續約2）')
                if '99-1 全球新興市場主動股票型（續約 2）' in page_text:
                    print('99-1 全球新興市場主動股票型（續約 2） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('99-1 全球新興市場主動股票型（續約 2）','99-1 全球新興市場股票型（續約2）')
                    LPF_LIST.append('99-1 全球新興市場股票型（續約2）')
                if '100-1 全球基本面指數被動股票型（續約 2）' in page_text:
                    print('100-1 全球基本面指數被動股票型（續約 2） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球基本面指數被動股票型（續約 2）','100-1 全球基本面指數被動股票型（續約2）')
                    LPF_LIST.append('100-1 全球基本面指數被動股票型（續約2）')
                if '100-1 全球不動產有價證券型（續約）' in page_text:
                    print('100-1 全球不動產有價證券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球不動產有價證券型（續約）','100-1 全球不動產有價證券型（續約）')
                    LPF_LIST.append('100-1 全球不動產有價證券型（續約）')
                if '100-1 全球不動產有價證券型（續約）' in page_text:
                    print('100-1 全球不動產有價證券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球不動產有價證券型（續約）','100-1 全球不動產有價證券型（續約）')
                    LPF_LIST.append('100-1 全球不動產有價證券型（續約）')
                if '100-1 全球不動產有價證券型（續約 2）' in page_text:
                    print('100-1 全球不動產有價證券型（續約 2） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球不動產有價證券型（續約 2）','100-1 全球不動產有價證券型（續約 2）')
                    LPF_LIST.append('100-1 全球不動產有價證券型（續約 2）')
                if '101-1 全球新興市場主動債券型（續約）' in page_text:
                    print('101-1 全球新興市場主動債券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101-1 全球新興市場主動債券型（續約）','101-1 全球新興市場主動債券型（續約）')
                    LPF_LIST.append('101-1 全球新興市場主動債券型（續約）')
                if '101-1 全球低波動指數被動股票型（續約）' in page_text:
                    print('101-1 全球低波動指數被動股票型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101-1 全球低波動指數被動股票型（續約））','101-1 全球低波動指數被動股票型（續約）')
                    LPF_LIST.append('101-1 全球低波動指數被動股票型（續約）')
                if '102-1 全球高股利增值股票型（續約）' in page_text:
                    print('102-1 全球高股利增值股票型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102-1 全球高股利增值股票型（續約）','102-1 全球高股利增值股票型(續約)')
                    LPF_LIST.append('102-1 全球高股利增值股票型(續約)')
                if '102-1 全球信用主動債券型  （續約）' in page_text:
                    print('102-1 全球信用主動債券型  （續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102-1 全球信用主動債券型  （續約）','102 - 1 全球信用主動債券型(續約)')
                    LPF_LIST.append('102 - 1 全球信用主動債券型(續約)')
                if '104-1 全球高品質被動股票型（續約）' in page_text:
                    print('104-1 全球高品質被動股票型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('104-1 全球高品質被動股票型（續約）','104-1 全球高品質被動股票型(續約)')
                    LPF_LIST.append('104-1 全球高品質被動股票型(續約)')
                if '104-2 全球基礎建設有價證券型（續約）' in page_text:
                    print('104-2 全球基礎建設有價證券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('104-2 全球基礎建設有價證券型（續約）','104-2 全球 基礎建設有價證券型(續約)')
                    LPF_LIST.append('104-2 全球 基礎建設有價證券型(續約)')            
                if '104-1 全球主權信用增值債券型  (續約)' in page_text:
                    print('104-1 全球主權信用增值債券型  (續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('104-1 全球主權信用增值債券型  (續約)','104-1 全球主權信用增值債券型(續約)')
                    LPF_LIST.append('104-1 全球主權信用增值債券型(續約)')  
                if '105-1 全球多元資產型  (續約)' in page_text:
                    print('105-1 全球多元資產型  (續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('105-1 全球多元資產型  (續約)','105-1 全球多元資產型 ( 續約 ) ')
                    LPF_LIST.append('105-1 全球多元資產型 ( 續約 ) ')  
                if '105-1 亞太混合指數增值股票型  (續約)' in page_text:
                    print('105-1 亞太混合指數增值股票型  (續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('105-1 亞太混合指數增值股票型  (續約)','105-1 亞太混合指數增值股票型(續約)')
                    LPF_LIST.append('105-1 亞太混合指數增值股票型(續約)') 
                if '106-1 全球 ESG 混合指數被動股票型' in page_text:
                    print('106-1 全球 ESG 混合指數被動股票型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106-1 全球 ESG 混合指數被動股票型','106-1 全球ESG混合指數被動股票型')
                    LPF_LIST.append('106-1 全球ESG混合指數被動股票型')
                if '106-1 絕對報酬債券型' in page_text:
                    print('106-1 絕對報酬債券型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106-1 絕對報酬債券型','106-1 絕對報酬債券型')
                    LPF_LIST.append('106-1 絕對報酬債券型')
                if '107-1 絕對報酬股票型' in page_text:
                    print('107-1 絕對報酬股票型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107-1 絕對報酬股票型','107-1絕對報酬股票型')
                    LPF_LIST.append('107-1絕對報酬股票型')
                if '108-1 全球新興市場動態多元因子指數增值股票型' in page_text:
                    print('108-1 全球新興市場動態多元因子指數增值股票型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('108-1 全球新興市場動態多元因子指數增值股票型','108-1全球新興市場動態多元因子指數增值股票型')
                    LPF_LIST.append('108-1全球新興市場動態多元因子指數增值股票型')
                if '109-1 全球美元公司增值債券型' in page_text:
                    print('109-1 全球美元公司增值債券型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('109-1 全球美元公司增值債券型','109-1全球美元公司增值債券型')
                    LPF_LIST.append('109-1全球美元公司增值債券型')
                if '110-1 全球基礎建設有價證券型' in page_text:
                    print('110-1 全球基礎建設有價證券型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110-1 全球基礎建設有價證券型','110-1全球基礎建設有價證券型')
                    LPF_LIST.append('110-1全球基礎建設有價證券型')
                if '110-1 全球多元資產型' in page_text:
                    print('110-1 全球多元資產型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110-1 全球多元資產型','110-1全球多元資產型')
                    LPF_LIST.append('110-1全球多元資產型')

            ###如果有交界處 這就很重要
            #elif p == 22:
                #print('到達23頁')
                if '110-1 全球多元資產型' in page_text:
                    print('110-1 全球多元資產型 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110-1 全球多元資產型','110-1全球多元資產型')
                    LPF_LIST.append('110-1全球多元資產型')
                #舊制，將PDF檔案中的基金名稱改成excel內的，過渡期
            elif p>=22:
                FUND_NAME = 'LRF'
                if '96 國際平衡型(續約 3)' in page_text:
                    print('96 國際平衡型(續約 3) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('96 國際平衡型(續約 3)','96 年續約3 國外委託經營 ( 平衡型  )')
                    LRF_LIST.append('96 年續約3 國外委託經營 ( 平衡型  )')
                if '97-1 全球增值債券型(續約 3)' in page_text:
                    print('97-1 全球增值債券型(續約 3)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-1 全球增值債券型(續約 3)','97年續約3國外委託經營( 全球增值債券型 )')
                    LRF_LIST.append('97年續約3國外委託經營( 全球增值債券型 )')      
            #elif p>=22:
                FUND_NAME = 'LRF'      
                if '97-1 全球增值股票型(續約 3)' in page_text:
                    print('97-1 全球增值股票型(續約 3)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-1 全球增值股票型(續約 3)','97年續約3 國外委託經營 ( 全球增值股票型 )')
                    LRF_LIST.append('97年續約3 國外委託經營 ( 全球增值股票型 )') 
                if '98-1 亞太(日本除外)主動股票型(續約 2)' in page_text:
                    print('98-1 亞太(日本除外)主動股票型(續約 2)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('98-1 亞太(日本除外)主動股票型(續約 2)','98-1 亞太（ 日本除外）股票型（ 續約2）')
                    LRF_LIST.append('98-1 亞太（ 日本除外）股票型（ 續約2）')
                if '100-1 全球基本面指數被動股票型(續約)' in page_text:
                    print('100-1 全球基本面指數被動股票型(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球基本面指數被動股票型(續約)','100年續約國外委託經營 ( 全球基本面指數股票型 )')
                    LRF_LIST.append('100年續約國外委託經營 ( 全球基本面指數股票型 )')
                    
                if '100-1 全球基本面指數被動股票型(續約 2) ' in page_text:
                    print('100-1 全球基本面指數被動股票型(續約 2) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球基本面指數被動股票型(續約 2) ','100-1 全球基本面指數被動股票型(續約 2) ')
                    LRF_LIST.append('100-1 全球基本面指數被動股票型(續約 2) ')

                if '101-1 全球低波動指數被動股票型(續約)' in page_text:
                    print('101-1 全球低波動指數被動股票型(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101-1 全球低波動指數被動股票型(續約)','101 年續約國外委託經營 (全球低波動指數股票型 )')
                    LRF_LIST.append('101 年續約國外委託經營 (全球低波動指數股票型 )')
                if '102-1 年全球高股利增值股票型(續約)' in page_text:
                    print('102-1 年全球高股利增值股票型(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102-1 年全球高股利增值股票型(續約)','102 年續約國外委託經營 ( 全球高股利增值股票型 )')
                    LRF_LIST.append('102 年續約國外委託經營 ( 全球高股利增值股票型 )')
                if '102-2 全球基礎建設有價證券型(續約)' in page_text:
                    print('102-2 全球基礎建設有價證券型(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102-2 全球基礎建設有價證券型(續約)','102 年續約國外委託經營 ( 全球基礎建設有價證券型 ) ')
                    LRF_LIST.append('102 年續約國外委託經營 ( 全球基礎建設有價證券型 ) ')
                if '102-2 全球不動產有價證券型 (續約)' in page_text:
                    print('102-2 全球不動產有價證券型 (續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102-2 全球不動產有價證券型 (續約)','102 年續約國外委託經營 ( 全球不動產有價證券型 ) ')
                    LRF_LIST.append('102 年續約國外委託經營 ( 全球不動產有價證券型 ) ')
                if '105-1 全球多元資產型 (續約)' in page_text:
                    print('105-1 全球多元資產型 (續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('105-1 全球多元資產型 (續約)','105-1 全球多元資產型 ( 續約 )')
                    LRF_LIST.append('105-1 全球多元資產型 ( 續約 )')
                if '106-1 全球 ESG 混合指數被動股票型' in page_text:
                    print('106-1 全球 ESG 混合指數被動股票型  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106-1 全球 ESG 混合指數被動股票型','106-1 全球ESG混合指數被動股票型')
                    LRF_LIST.append('106-1 全球ESG混合指數被動股票型')
                if '106-1 絕對報酬債券型' in page_text:
                    print('106-1 絕對報酬債券型  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106-1 絕對報酬債券型','106-1 絕對報酬債券型')
                    LRF_LIST.append('106-1 絕對報酬債券型')
                if '107-1 絕對報酬股票型' in page_text:
                    print('107-1 絕對報酬股票型  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107-1 絕對報酬股票型','107-1絕對報酬股票型')
                    LRF_LIST.append('107-1絕對報酬股票型')
                if '110-1 全球基礎建設有價證券型' in page_text:
                    print('110-1 全球基礎建設有價證券型  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110-1 全球基礎建設有價證券型','110-1全球基礎建設有價證券型')
                    LRF_LIST.append('110-1全球基礎建設有價證券型')
            
            for target_fund in list(LPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p<22:
                            有新制勞退目標基金的頁數.append((p,table_num))
                            新制勞退目標基金的名稱.append(target_fund)
                            NEW_FUND_NAMES.append(FUND_NAME)
                            table_num+=1
            for target_fund in list(LRF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p>=22:
                            有舊制勞退目標基金的頁數.append((p,table_num))
                            舊制勞退目標基金的名稱.append(target_fund)
                            OLD_FUND_NAMES.append(FUND_NAME)
                            table_num+=1


        #這邊比較特別，因為有發現表格較為混雜的情況，所以這邊的作法是找到所有目標表格在地頁數，後續用try except找到可以處理的表格
        LPF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_f)
        table_name_index = 0
        ps = PDF_file.pages
        for page,table_num in 有新制勞退目標基金的頁數:
            print(ps)
            print(page)
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_新勞退 = pd.DataFrame(page_tables[table_num])
                #處理表格內的資料
                table_新勞退.iloc[0] = table_新勞退.iloc[0].fillna('',axis=0)
                table_新勞退.iloc[1] = table_新勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_新勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_新勞退.iloc[0,4]).group(2)
                table_新勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_新勞退.iloc[0,4])
                table_新勞退_final = table_新勞退.iloc[2:,:]
                table_新勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_新勞退_final['Target'] = table_新勞退_final['Target'].fillna(method = 'ffill')
                table_新勞退_final[['Fund','Mandate Type','From','As of']]=['LPF',新制勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                table_新勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_新勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_新勞退_final = table_新勞退_final[table_新勞退_final['SITE']!='合計']
                LPF = LPF.append(table_新勞退_final)
                table_name_index += 1
            except:
                print(str(page)+'有問題')
                table_name_index += 0
        LPF.reset_index(drop=True,inplace=True)
        print('新制勞退處理完畢')

        LRF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_f)
        old_table_name_index = 0
        ps = PDF_file.pages
        for old_page,old_table_num in 有舊制勞退目標基金的頁數:
            pg = ps[old_page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_舊勞退 = pd.DataFrame(page_tables[old_table_num])
                #處理表格內的資料
                table_舊勞退.iloc[0] = table_舊勞退.iloc[0].fillna('',axis=0)
                table_舊勞退.iloc[1] = table_舊勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_舊勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_舊勞退.iloc[0,4]).group(2)
                table_舊勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_舊勞退.iloc[0,4])
                table_舊勞退_final = table_舊勞退.iloc[2:,:]
                table_舊勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_舊勞退_final['Target'] = table_舊勞退_final['Target'].fillna(method = 'ffill')
                table_舊勞退_final[['Fund','Mandate Type','From','As of']]=['LRF',舊制勞退目標基金的名稱[old_table_name_index],Date_Start,Date_End]
                table_舊勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_舊勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_舊勞退_final = table_舊勞退_final[table_舊勞退_final['SITE']!='合計']
                LRF = LRF.append(table_舊勞退_final)
                old_table_name_index += 1
            except:
                print(str(page)+'有問題')
                old_table_name_index += 0
        LRF.reset_index(drop=True,inplace=True)
        print('舊制勞退處理完畢')

97-1 全球主動債券型（續約 3） in 14頁
97-2 全球增值債券型（續約 3） in 14頁
97-2 全球增值股票型（續約 3） in 15頁
98-1 亞太（日本除外）主動股票型（續約 2） in 15頁
98-1 全球被動股票型（續約 2） in 15頁
99-1 全球新興市場主動股票型（續約 2） in 15頁
100-1 全球基本面指數被動股票型（續約 2） in 16頁
100-1 全球不動產有價證券型（續約 2） in 16頁
101-1 全球新興市場主動債券型（續約） in 16頁
101-1 全球低波動指數被動股票型（續約） in 17頁
102-1 全球高股利增值股票型（續約） in 17頁
102-1 全球信用主動債券型  （續約） in 17頁
104-1 全球高品質被動股票型（續約） in 18頁
104-2 全球基礎建設有價證券型（續約） in 18頁
104-1 全球主權信用增值債券型  (續約) in 18頁
105-1 全球多元資產型  (續約) in 19頁
105-1 亞太混合指數增值股票型  (續約) in 19頁
106-1 全球 ESG 混合指數被動股票型 in 19頁
106-1 絕對報酬債券型 in 20頁
107-1 絕對報酬股票型 in 20頁
108-1 全球新興市場動態多元因子指數增值股票型 in 21頁
109-1 全球美元公司增值債券型 in 21頁
110-1 全球基礎建設有價證券型 in 22頁
110-1 全球多元資產型 in 22頁
96 國際平衡型(續約 3) in 23頁
97-1 全球增值債券型(續約 3)  in 23頁
97-1 全球增值股票型(續約 3)  in 23頁
98-1 亞太(日本除外)主動股票型(續約 2)  in 24頁
100-1 全球基本面指數被動股票型(續約 2) in 24頁
101-1 全球低波動指數被動股票型(續約)  in 24頁
102-1 年全球高股利增值股票型(續約)  in 25頁
102-2 全球基礎建設有價證券型(續約)  in 25頁
102-2 全球不動產有價證券型 (續約)  in 25頁
105-1 全球多元資產型 (續約)  in 26頁
106-1 全球 ESG 混合指數被動股票型  in 26頁
106-1 絕對報酬債券型 

In [62]:
有舊制勞退目標基金的頁數

[(22, 0),
 (22, 1),
 (22, 2),
 (23, 0),
 (23, 1),
 (23, 2),
 (24, 0),
 (24, 1),
 (24, 2),
 (25, 0),
 (25, 1),
 (25, 2),
 (26, 0),
 (26, 1)]

In [14]:
print(len(有舊制勞退目標基金的頁數))
print(len(舊制勞退目標基金的名稱))

11
11


### NPIF 國保基金

In [17]:
if '國民年金保險' in PDF_file and '.pdf' in PDF_file:
    國保_PDF = pr.open(input_path+'/'+PDF_file)
#從0開始
#是一個list
ps              = 國保_PDF.pages
pg = ps[4]
print('104-1 全球高品質被動股票型(續約)' in pg.extract_text())

True


In [29]:
NPIF_LIST = []
有國保目標基金的頁數 = []
國保目標基金的名稱   = []
NPIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '國民年金保險' in PDF_file and '.pdf' in PDF_file:
        print(PDF_file)
        國保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 國保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'NPIF'
            if '104-1 全球高品質被動股票型(續約)' in page_text:
                print('104-1 全球高品質被動股票型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-1 全球高品質被動股票型(續約)','104-1 全球高品質被動股票型(續約)')
                NPIF_LIST.append('104-1 全球高品質被動股票型(續約)')
            if '104-2 全球不動產有價證券型(續約)' in page_text:
                print('104-2 全球不動產有價證券型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-2 全球不動產有價證券型(續約)','104-2 全球不動產有價證券型(續約)')
                NPIF_LIST.append('104-2 全球不動產有價證券型(續約)')
            if '104-1 全球主權信用增值債券型(續約) ' in page_text:
                print('104-1 全球主權信用增值債券型(續約)  in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-1 全球主權信用增值債券型(續約) ','104-1 全球主權信用增值債券型 (續約)') 
                NPIF_LIST.append('104-1 全球主權信用增值債券型 (續約)')
            if '105-1 亞太混合指數增值股票型(續約) ' in page_text:
                print('105-1 亞太混合指數增值股票型(續約)  in ' + str(p+1) + '頁')
                page_text = page_text.replace('105-1 亞太混合指數增值股票型(續約) ','105-1 亞太混合指數增值股票型(續約)')  
                NPIF_LIST.append('105-1 亞太混合指數增值股票型(續約)')              
            if '106-1 全球 ESG 混合指數被動股票型' in page_text:
                print('106-1 全球 ESG 混合指數被動股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('106-1 全球 ESG 混合指數被動股票型','106-1 全球ESG混合指數被動股票型')
                NPIF_LIST.append('106-1 全球ESG混合指數被動股票型')                
            if '106-1 絕對報酬債券型 ' in page_text:
                print('106-1 絕對報酬債券型  in ' + str(p+1) + '頁')
                page_text = page_text.replace('106-1 絕對報酬債券型 ','106-1 絕對報酬債券型')
                NPIF_LIST.append('106-1 絕對報酬債券型')                
            if '107-1 絕對報酬股票型' in page_text:
                print('107-1 絕對報酬股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('107-1 絕對報酬股票型','107-1絕對報酬股票型') 
                NPIF_LIST.append('107-1絕對報酬股票型')               
            if '109-1 全球美元公司增值債券型' in page_text:
                print('109-1 全球美元公司增值債券型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('109-1 全球美元公司增值債券型','109-1全球美元公司增值債券型')
                NPIF_LIST.append('109-1全球美元公司增值債券型')
            if '110-1 全球多元資產型' in page_text:
                print('110-1 全球多元資產型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('110-1 全球多元資產型','110-1全球多元資產型')
                NPIF_LIST.append('110-1全球多元資產型')

            for target_fund in list(NPIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有國保目標基金的頁數.append((p,table_num))
                        國保目標基金的名稱.append(target_fund)
                        NPIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1
                            
        NPIF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_file)
        NPIF_table_name_index = 0
        ps = PDF_file.pages
        for NPIF_page,NPIF_table_num in 有國保目標基金的頁數:
            pg = ps[NPIF_page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_NPIF = pd.DataFrame(page_tables[NPIF_table_num])
                #處理表格內的資料
                table_NPIF.iloc[0] = table_NPIF.iloc[0].fillna('',axis=0)
                table_NPIF.iloc[1] = table_NPIF.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_NPIF.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_NPIF.iloc[0,4]).group(2)
                table_NPIF.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_NPIF.iloc[0,4])
                table_NPIF_final = table_NPIF.iloc[2:,:]
                table_NPIF_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_NPIF_final['Target'] = table_NPIF_final['Target'].fillna(method = 'ffill')
                table_NPIF_final[['Fund','Mandate Type','From','As of']]=[FUND_NAME,國保目標基金的名稱[NPIF_table_name_index],Date_Start,Date_End]
                table_NPIF_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_NPIF_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_NPIF_final = table_NPIF_final[table_NPIF_final['SITE']!='合計']
                NPIF = NPIF.append(table_NPIF_final)
                NPIF_table_name_index += 1
            except:
                print(str(page)+'有問題')
                NPIF_table_name_index += 0
        NPIF.reset_index(drop=True,inplace=True)
            


國民年金保險基金-111年4月.pdf
104-1 全球高品質被動股票型(續約) in 5頁
104-2 全球不動產有價證券型(續約) in 5頁
104-1 全球主權信用增值債券型(續約)  in 5頁
105-1 亞太混合指數增值股票型(續約)  in 6頁
106-1 全球 ESG 混合指數被動股票型 in 6頁
106-1 絕對報酬債券型  in 6頁
107-1 絕對報酬股票型 in 7頁
109-1 全球美元公司增值債券型 in 7頁
110-1 全球多元資產型 in 8頁


In [30]:
有國保目標基金的頁數

[(4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (7, 0)]

In [31]:
NPIF

,SITE,Mandate Size,AUM,YTD %,Since Inception %,Ranking,Target,Fund,Mandate Type,From,As of
0,北美信託,"279,504,715","325,403,514",-16.63,16.42,2,15.83,NPIF,104-1 全球高品質被動股票型(續約),2020/08/11,2022/04/30
1,LGIM,"279,205,663","324,659,771",-16.57,16.28,3,15.83,NPIF,104-1 全球高品質被動股票型(續約),2020/08/11,2022/04/30
2,Geode,"278,841,018","324,946,728",-16.69,16.53,1,15.83,NPIF,104-1 全球高品質被動股票型(續約),2020/08/11,2022/04/30
3,CBRE,"115,859,702","162,607,524",-9.69,40.35,1,33.18,NPIF,104-2 全球不動產有價證券型(續約),2020/09/25,2022/04/30
4,信安環球,"170,968,577","218,392,848",-10.00,27.74,2,33.18,NPIF,104-2 全球不動產有價證券型(續約),2020/09/25,2022/04/30
5,盧米斯,"294,532,220","247,673,703",-11.83,-15.91,－,-18.27,NPIF,104-1 全球主權信用增值債券型 (續約),2020/11/13,2022/04/30
6,貝萊德,"151,227,051","136,741,706",-10.49,-9.58,－,-9.39,NPIF,105-1 亞太混合指數增值股票型(續約),2021/08/24,2022/04/30
7,貝萊德,"125,000,000","171,046,868",-10.46,50.03,4,49.35,NPIF,106-1 全球ESG混合指數被動股票型,2017/06/22,2022/04/30
8,DWS,"125,000,000","171,276,913",-10.68,50.41,3,49.35,NPIF,106-1 全球ESG混合指數被動股票型,2017/06/22,2022/04/30
9,道富,"125,000,000","171,573,609",-10.62,51.06,1,49.35,NPIF,106-1 全球ESG混合指數被動股票型,2017/06/22,2022/04/30


### 退撫

In [18]:
PSPF_LIST = []
有退撫目標基金的頁數 = []
退撫目標基金的名稱   = []
PSPF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file and '.pdf' in PDF_file:
        退撫_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 退撫_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'PSPF'
            if '98年-2續約亞太股票型' in page_text:
                print('98年-2續約亞太股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('98年-2續約亞太股票型','第 5 次續約 - 亞太股票型')
                PSPF_LIST.append('第 5 次續約 - 亞太股票型')
            if '98年-2續約國際股票型' in page_text:
                print('98年-2續約國際股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('98年-2續約國際股票型','第 5 次續約 - 國際股票型')
                PSPF_LIST.append('第 5 次續約 - 國際股票型')
            if '100年-1國際股票型' in page_text:
                print('100年-1國際股票型  in ' + str(p+1) + '頁')
                page_text = page_text.replace('100年-1國際股票型','第 6 次 - 國際股票型') 
                PSPF_LIST.append('第 6 次 - 國際股票型')
            if '100年-2公司債券型' in page_text:
                print('100年-2公司債券型  in ' + str(p+1) + '頁')
                page_text = page_text.replace('100年-2公司債券型','第 7 次 - 公司債券型')    
                PSPF_LIST.append('第 7 次 - 公司債券型')            
            if '102年高股利股票型' in page_text:
                print('102年高股利股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('102年高股利股票型','第 9 次 - 高股利股票型')  
                PSPF_LIST.append('第 9 次 - 高股利股票型')       
            if '102年低波動股票型' in page_text:
                print('102年低波動股票型  in ' + str(p+1) + '頁')
                page_text = page_text.replace('102年低波動股票型','第 9 次 - 低波動股票型') 
                PSPF_LIST.append('第 9 次 - 低波動股票型')   
            if '103年基礎建設股票型' in page_text:
                print('103年基礎建設股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('103年基礎建設股票型','第 10 次 - 基礎建設股票型') 
                PSPF_LIST.append('第 10 次 - 基礎建設股票型')  
            if '103年不動產股票型' in page_text:
                print('103年不動產股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('103年不動產股票型','第 10 次 - 不動產股票型') 
                PSPF_LIST.append('第 10 次 - 不動產股票型')                
            if '105年多元資產型' in page_text:
                print('105年多元資產型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('105年多元資產型','第 11 次 - 多元資產型')
                PSPF_LIST.append('第 11 次 - 多元資產型')
            if '106年總報酬固定收益型' in page_text:
                print('106年總報酬固定收益型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('106年總報酬固定收益型','第12次-總報酬固定收益型')
                PSPF_LIST.append('第12次-總報酬固定收益型')
            if '110年高品質ESG股票型' in page_text:
                print('110年高品質ESG股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('110年高品質ESG股票型','第13次-高品質ESG 股票型')
                PSPF_LIST.append('第13次-高品質ESG 股票型')
            for target_fund in list(PSPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有退撫目標基金的頁數.append((p,table_num))
                        退撫目標基金的名稱.append(target_fund)
                        PSPF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1
        


        PSPF = pd.DataFrame({})
        PSPF_table_name_index = 0
        for PSPF_page,PSPF_table_num in 有退撫目標基金的頁數:
            pg = ps[PSPF_page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_PSPF     = pd.DataFrame(page_tables[PSPF_table_num])
                multi          = [table_PSPF.iloc[0,:].dropna(axis=0),table_PSPF.iloc[1,:].fillna('')+table_PSPF.iloc[2,:].fillna('')]
                mindex         = pd.MultiIndex.from_product(multi,names=['Level1','Level2'])
                table_最終結果  = pd.DataFrame(columns = mindex, index = table_PSPF.iloc[3:,:].index)
                table_最終結果[table_PSPF.iloc[0,:].dropna(axis=0)[0]] = table_PSPF.iloc[3:,:].values

                #將資料補齊
                #找到基金的日期
                Funds_Date     = table_最終結果.columns.get_level_values(0)[0]
                table_最終結果  = table_最終結果[table_最終結果[Funds_Date,'受託機構名稱'] != '合計']
                table_最終結果[Funds_Date,'Fund'] = FUND_NAME
                table_最終結果[Funds_Date,'Mandate Type'] = 退撫目標基金的名稱[PSPF_table_name_index]
                for x in table_最終結果.columns.get_level_values(1):
                    if '指定' in x:
                        #print(x)
                        table_最終結果[Funds_Date,x.replace('\n','')] = table_最終結果[Funds_Date,x]
                #將報酬率補起來，有的是目標，有的是指標
                table_最終結果[Funds_Date,'指定指標報酬率(%)'] = table_最終結果[Funds_Date,'指定指標報酬率(%)'].fillna(method = 'ffill')
                table_最終結果[Funds_Date,'受託機構名稱'] = table_最終結果[Funds_Date,'受託機構名稱'].str.replace('投信','')
                #補開始及結束的日期
                table_最終結果[Funds_Date,'From']  = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][3:])).replace('日','')
                table_最終結果[Funds_Date,'As of'] = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][3:])).replace('日','')
                for target in ['年','月']:
                    table_最終結果[Funds_Date,'From']  = table_最終結果[Funds_Date,'From'].str.replace(target,'/')
                    table_最終結果[Funds_Date,'As of'] = table_最終結果[Funds_Date,'As of'].str.replace(target,'/')
                PSPF = PSPF.append(pd.DataFrame(table_最終結果.values))
                PSPF_table_name_index += 1
            except:
                print(str(page)+'有問題')
                PSPF_table_name_index += 0
        PSPF.reset_index(drop=True,inplace=True)
        PSPF = PSPF.iloc[:,[0,1,2,3,4,5,7,8,9,10,11]]

PSPF.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Fund','Mandate Type','Target','From','As of']

98年-2續約亞太股票型 in 7頁
98年-2續約國際股票型 in 7頁
100年-1國際股票型  in 7頁
100年-2公司債券型  in 7頁
102年高股利股票型 in 8頁
102年低波動股票型  in 8頁
103年基礎建設股票型 in 8頁
103年不動產股票型 in 8頁
105年多元資產型 in 9頁
106年總報酬固定收益型 in 9頁
110年高品質ESG股票型 in 9頁


### LIF

In [19]:
LIF_LIST = []
有勞保目標基金的頁數 = []
勞保目標基金的名稱   = []
LIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '勞工保險' in PDF_file and '.pdf' in PDF_file:
        勞保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'LIF'
            if '96 年全球新興市場主動股票型(續約 2)' in page_text:
                print('96 年全球新興市場主動股票型(續約 2) in ' + str(p+1) + '頁')
                page_text = page_text.replace('96 年全球新興市場主動股票型(續約 2)','96 年全球新興市場主動股票型 (續約)')
                LIF_LIST.append('96 年全球新興市場主動股票型 (續約)')
            if '100 年全球主動股票型(續約 2)' in page_text:
                print('100 年全球主動股票型(續約 2) in ' + str(p+1) + '頁')
                page_text = page_text.replace('100 年全球主動股票型(續約 2)','100 年全球主動股票型 ( 續約 )')
                LIF_LIST.append('100 年全球主動股票型 ( 續約 )')
            if '101 年全球新興市場主動債券型(續約)' in page_text:
                print('101 年全球新興市場主動債券型(續約)  in ' + str(p+1) + '頁')
                page_text = page_text.replace('101 年全球新興市場主動債券型(續約)','101 年全球新興市場主動債券型 ( 續約 )') 
                LIF_LIST.append('101 年全球新興市場主動債券型 ( 續約 )')       
            if '104 年全球主動債券型(續約 2)' in page_text:
                print('104 年全球主動債券型(續約 2)  in ' + str(p+1) + '頁')
                page_text = page_text.replace('104 年全球主動債券型(續約 2)','104 年全球主動債券型 ( 續約2 )')     
                LIF_LIST.append('104 年全球主動債券型 ( 續約2 )')
            if '104-1 全球高品質被動股票型(續約)' in page_text:
                print('104-1 全球高品質被動股票型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-1 全球高品質被動股票型(續約)','104-1 全球高品質被動股票型(續約)') 
                LIF_LIST.append('104-1 全球高品質被動股票型(續約)')     
            if '104-2 全球基礎建設有價證券型(續約)' in page_text:
                print('104-2 全球基礎建設有價證券型(續約)  in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-2 全球基礎建設有價證券型(續約)','104-2 全球基礎建設有價證券型(續約)') 
                LIF_LIST.append('104-2 全球基礎建設有價證券型(續約)')  
            if '104-2 全球不動產有價證券型(續約)' in page_text:
                print('104-2 全球不動產有價證券型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-2 全球不動產有價證券型(續約)','104-2 全球不動產有價證券型(續約)')  
                LIF_LIST.append('104-2 全球不動產有價證券型(續約)') 
            if '104-1 全球主權信用增值債券型(續約)' in page_text:
                print('104-1 全球主權信用增值債券型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('104-1 全球主權信用增值債券型(續約)','104-1 全球主權信用增值債券型(續約)')  
                LIF_LIST.append('104-1 全球主權信用增值債券型(續約)')   
            if '105-1 全球多元資產型(續約)' in page_text:
                print('105-1 全球多元資產型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('105-1 全球多元資產型(續約)','105-1 全球多元資產型 ( 續約 )')
                LIF_LIST.append('105-1 全球多元資產型 ( 續約 )') 
            if '105-1 亞太混合指數增值股票型(續約)' in page_text:
                print('105-1 亞太混合指數增值股票型(續約) in ' + str(p+1) + '頁')
                page_text = page_text.replace('105-1 亞太混合指數增值股票型(續約)','105-1 亞太混合指數增值股票型(續約)')
                LIF_LIST.append('105-1 亞太混合指數增值股票型(續約)')
            if '106-1 全球 ESG 混合指數被動股票型' in page_text:
                print('106-1 全球 ESG 混合指數被動股票型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('106-1 全球 ESG 混合指數被動股票型','106-1 全球ESG混合指數被動股票型')
                LIF_LIST.append('106-1 全球ESG混合指數被動股票型')
            if '106-1 絕對報酬債券型' in page_text:
                print('106-1 絕對報酬債券型 in ' + str(p+1) + '頁')
                page_text = page_text.replace('106-1 絕對報酬債券型','106-1 絕對報酬債券型')
                LIF_LIST.append('106-1 絕對報酬債券型')
                
            page_text = page_text.replace('\n','')
            for target_fund in list(LIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有勞保目標基金的頁數.append((p,table_num))
                        勞保目標基金的名稱.append(target_fund)
                        LIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

        LIF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_file)
        LIF_table_name_index = 0
        ps = PDF_file.pages
        for LIF_page,LIF_table_num in 有勞保目標基金的頁數:
            pg = ps[LIF_page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_LIF = pd.DataFrame(page_tables[LIF_table_num])
                #處理表格內的資料
                table_LIF.iloc[0] = table_LIF.iloc[0].fillna('',axis=0)
                table_LIF.iloc[1] = table_LIF.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_LIF.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_LIF.iloc[0,4]).group(2)
                table_LIF.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_LIF.iloc[0,4])
                table_LIF_final = table_LIF.iloc[2:,:]
                table_LIF_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_LIF_final['Target'] = table_LIF_final['Target'].fillna(method = 'ffill')
                table_LIF_final[['Fund','Mandate Type','From','As of']]=[FUND_NAME,勞保目標基金的名稱[LIF_table_name_index],Date_Start,Date_End]
                table_LIF_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_LIF_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_LIF_final = table_LIF_final[table_LIF_final['SITE']!='合計']
                LIF = LIF.append(table_LIF_final)
                LIF_table_name_index += 1
            except:
                print(str(page)+'有問題')
                LIF_table_name_index += 0
        LIF = LIF[LIF['SITE']!='']
        LIF.reset_index(drop=True,inplace=True)
        

96 年全球新興市場主動股票型(續約 2) in 5頁
100 年全球主動股票型(續約 2) in 5頁
101 年全球新興市場主動債券型(續約)  in 5頁
104 年全球主動債券型(續約 2)  in 6頁
104-1 全球高品質被動股票型(續約) in 6頁
104-2 全球基礎建設有價證券型(續約)  in 6頁
104-2 全球不動產有價證券型(續約) in 7頁
104-1 全球主權信用增值債券型(續約) in 7頁
105-1 全球多元資產型(續約) in 7頁
105-1 亞太混合指數增值股票型(續約) in 7頁
106-1 全球 ESG 混合指數被動股票型 in 8頁
106-1 絕對報酬債券型 in 8頁


# 後續動作

In [63]:
#做中英文轉換
convert_dict = {
    '聯博':'Alliance Bernstein',
    '富蘭克林坦伯頓':'Franklin',
    '富蘭克林 坦伯頓':'Franklin',#這個要固定
    'PIMCO':'PIMCO',
    '貝萊德':'BlackRock',
    '摩根':'J.P. Morgan',
    '景順':'Invesco',
    '道富':'State Street',
    '瑞萬通博':'Vontobel',
    'Cohen & Steers':'Cohen & Steers',
    'Ashmore':'Ashmore',
    'Pictet':'Pictet',
    'MFS':'MFS',
    '威靈頓':'Wellington',
    'Bluebay':'Bluebay',
    '盧米斯':'Loomis Sayles',
    '北美信託':'Northern Trust',
    'LGIM':'LGIM',
    'Geode':'Geode',
    'Magellan':'Magellan',
    'RREEF':'RREEF',
    'Brandywine':'Brandywine',
    'Allianz':'Allianz',
    'DWS':'DWS',
    '柏瑞':'PineBridge',
    'American Century':'American Century',
    '富達':'Fidelity',
    'TCW':'TCW',
    'Nomura':'Nomura',
    'CPR':'CPR',
    'Amundi':'Amundi',
    '荷寶':'Robeco',
    'INSIGHT':'INSIGHT',
    'WESTERN':'Western',
    'CBRE':'CBRE',
    'Clearbridge':'ClearBridge',
    'Ninety One':'Ninety one',
    'Schroders':'Schroders',
    'T. Rowe Price':'T. Rowe Price',
    '瑞銀':'UBS',
    'Lazard':'Lazard',
    '信安 環球':'PGI',
    'AMP':'AMP',
    'ClearBridge':'ClearBridge',
    '麥格理':'Macquarie',
    '信安環球':'PGI',
    'JP Morgan':'J.P. Morgan',
    'Ninety one':'Ninety one',
    '富達國際':'Fidelity',
    '安聯環球投資':'Allianz',
    '施羅德投資管理':'Schroders',
    '東方匯理資產管理':'Amundi',
    '貝萊德法人信託':'BlackRock',
    '瑞銀資產管理':'UBS',
    '德意志資產管理':'DWS',
    '摩根資產管理':'J.P. Morgan',
    '貝萊德財務管理':'BlackRock',
    'INSIGHT':'Insight'
}
for x in [LPF,LRF,LIF,NPIF,PSPF]:
    #x.drop(columns = ['Target'],inplace=True)
    x['SITE'] = x['SITE'].str.replace('\n','')
    x['Asset Manager'] = x['SITE'].map(convert_dict)
LPF.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/LPF.csv',encoding='utf-8-sig',header=True,index=False)
LRF.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/LRF.csv',encoding='utf-8-sig',header=True,index=False)
LIF.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/LIF.csv',encoding='utf-8-sig',header=True,index=False)
NPIF.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/NPIF.csv',encoding='utf-8-sig',header=True,index=False)
PSPF.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/PSPF.csv',encoding='utf-8-sig',header=True,index=False)

### 合併

In [64]:
GFI['新舊'] = 'old'

In [65]:
Final_df = pd.concat([LPF,LRF,LIF,NPIF,PSPF],axis=0)
Final_df.reset_index(drop=True,inplace=True)
Final_df['SITE'] = Final_df['SITE'].str.replace('\n','')
Final_df['Asset Manager'] = Final_df['SITE'].map(convert_dict)
Final_df = Final_df[['Fund','Mandate Type','Asset Manager','Mandate Size','AUM','YTD %','Since Inception %','Ranking','From','As of']]
Final_df.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/page3_final_check.csv',encoding='UTF-8-SIG',index=False,header=True)

In [66]:
新舊合併 = pd.concat([Final_df,GFI],axis=0)
新舊合併 = 新舊合併[pd.isna(新舊合併['Fund'])==False]
for col in ['Fund','Mandate Type','Asset Manager']:
    新舊合併[col] = 新舊合併[col].str.strip()
新舊合併_去重複 = 新舊合併.drop_duplicates(subset = ['Fund','Mandate Type','Asset Manager'],keep = 'first')
新舊合併_去重複.reset_index(drop=True,inplace=True)

In [67]:
新舊合併_去重複.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/page3_下半部有old標記為可能已刪除的.csv',encoding='UTF-8-SIG',index=False,header=True)

In [68]:
新舊合併_去重複_new = 新舊合併_去重複[新舊合併_去重複['新舊'] != 'old']

In [69]:
GFI = GFI[pd.isna(GFI['Fund'])==False]

In [76]:
新舊合併_去重複_new

,Fund,Mandate Type,Asset Manager,Mandate Size,AUM,YTD %,Since Inception %,Ranking,From,As of,Category,Investment Type,Asset Class,新舊
0,LPF,97-1 全球債券型（ 續約3）,Alliance Bernstein,"907,613,021","775,930,530",-12.19,-14.51,2,2021/6/7,2022/4/30,Global,Active,Fixed Income,NaN
1,LPF,97-1 全球債券型（ 續約3）,Franklin,"851,737,838","737,854,426",-9.24,-13.37,1,2021/6/7,2022/4/30,NaN,NaN,NaN,NaN
2,LPF,97-2 全球增值債券型（續約 3）,PIMCO,"1,858,967,886","1,658,351,472",-10.98,-10.98,－,2022/1/6,2022/4/30,NaN,NaN,NaN,NaN
3,LPF,97-2 全球增值股票型（續約 3）,BlackRock,"1,734,805,538","1,588,135,837",-8.45,-8.45,2,2022/2/6,2022/4/30,NaN,NaN,NaN,NaN
4,LPF,97-2 全球增值股票型（續約 3）,J.P. Morgan,"1,485,120,883","1,359,570,629",-8.45,-8.45,1,2022/2/6,2022/4/30,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,PSPF,第12次-總報酬固定收益型,DWS,"350,000,000","357,048,104.31",-7.72,6.77,3,2018/6/5,2022/4/30,NaN,NaN,NaN,NaN
177,PSPF,第12次-總報酬固定收益型,TCW,"350,000,000","367,912,712.13",-4.54,10.08,2,2018/6/5,2022/4/30,NaN,NaN,NaN,NaN
178,PSPF,第12次-總報酬固定收益型,American Century,"250,000,000","259,412,397.71",-4.11,6.49,4,2018/6/5,2022/4/30,NaN,NaN,NaN,NaN
179,PSPF,第13次-高品質ESG 股票型,BlackRock,"200,000,000","175,630,169.67",-14.17,-10.65,2,2021/9/27,2022/4/30,NaN,NaN,NaN,NaN


In [81]:
#先skip
GFI['Asset Manager'] = GFI['Asset Manager'].astype('str')
新舊合併_去重複_new['Asset Manager'] = 新舊合併_去重複_new['Asset Manager'].astype('str')
a = 0
match  = []
for x in range(len(新舊合併_去重複_new)):
    for y in range(len(GFI)):
        if 新舊合併_去重複_new.loc[x,'Fund'].strip().lower() == GFI.loc[y,'Fund'].strip().lower() and 新舊合併_去重複_new.loc[x,'Mandate Type'].strip().lower() == GFI.loc[y,'Mandate Type'].strip().lower() and 新舊合併_去重複_new.loc[x,'Asset Manager'].strip().lower() == GFI.loc[y,'Asset Manager'].strip().lower():
            #排名不更新
            a+=1
            match.append((x,y))
            if 新舊合併_去重複_new.loc[x,'Ranking'] == '－' or 新舊合併_去重複_new.loc[x,'Ranking'] == '–' or 新舊合併_去重複_new.loc[x,'Ranking'] == '-' or Final_df.loc[x,'Ranking'] == '－':
                print('ranking no change')
                要更新的資料 = list(GFI.loc[y,['Category','Investment Type','Asset Class','Ranking']].values)
                新舊合併_去重複_new.loc[x,['Category','Investment Type','Asset Class','Ranking']] = 要更新的資料
            else:
                print('ranking change')
                要更新的資料 = list(GFI.loc[y,['Category','Investment Type','Asset Class']].values)
                新舊合併_去重複_new.loc[x,['Category','Investment Type','Asset Class']] = 要更新的資料
        else:
            pass
            

ranking change
ranking no change
ranking change
ranking change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking no change
ranking change
ranking change
ranking no change
ranking no

In [82]:
新舊合併_去重複_new.to_csv(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/output/page3.csv',encoding='UTF-8-SIG',index=False,header=True)